### Integração Numérica - Regra de 1/3 de Simpson

Este notebook apresenta a implementação da Regra de 1/3 de Simpson para o cálculo de integrais definidas, com cálculo do erro de aproximação simples e generalizado, utilizando orientação a objetos com uma classe base comum.


In [1]:
%load_ext autoreload
%autoreload 2
from integral import Integral
import math

### Classe `IntegralUmTercoSimpson`

In [6]:
class IntegralUmTercoSimpson(Integral):
    def __init__(self, x=None, y=None, limite_inf=None, limite_sup=None, func_integral=None, h=None):
        super().__init__()
        self._setup(x, y, limite_inf, limite_sup, func_integral, h)

        # Ajustar se pontos não múltiplos de 2 (n deve ser par)
        # if (len(self.x) - 1) % 2 != 0:
            # raise ValueError("Para Simpson 1/3, número de intervalos deve ser par (n deve ser múltiplo de 2).")

    def integral(self):
        n = len(self.x) - 1
        soma = self.y[0] + self.y[-1]

        for i in range(1, n):
            if i % 2 == 0:
                soma += 2 * self.y[i]
            else:
                soma += 4 * self.y[i]

        return (self.h / 3) * soma

    def erro_simples(self):
        d4_inf = self.derivada_quarta_ordem(self.limite_inf)
        d4_sup = self.derivada_quarta_ordem(self.limite_sup)
        maior = self.saber_maior(d4_inf, d4_sup)
        parcela_1 = ((self.h ** 4) * (self.limite_sup - self.limite_inf)) / 180
        return abs(maior * parcela_1)

    def erro_generalizado(self):
        return self.erro_simples()

### Exemplo de uso com pontos

In [16]:
# Questão velocidade - Lista

pontos_x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
pontos_y = [0, 50.4, 53.7, 67.5, 74.3, 84.6, 92.1, 98.3, 100, 105, 110]

simpson_13 = IntegralUmTercoSimpson(x=pontos_x, y=pontos_y)
print(f"Simpson 1/3 Margem 1 (x, y): {simpson_13.integral():.5f}")
print(f"Erro simples: {simpson_13.erro_simples():.5f}")
print(f"Erro generalizado: {simpson_13.erro_generalizado():.5f}")

print(f"Simpson 1/3 Margem M2 resultante (x, y): {(simpson_13.integral() + 2665.00000):.5f}")


Simpson 1/3 Margem 1 (x, y): 791.13333
Erro simples: 488.54093
Erro generalizado: 488.54093
Simpson 1/3 Margem M2 resultante (x, y): 3456.13333


### Exemplo de uso com função

In [17]:
# Questão 2: integral de 0 a 1 3*exp(-x) - Lista
def f(x):
    return 3 * math.exp(-x)

simpson_13_2 = IntegralUmTercoSimpson(limite_inf=0, limite_sup=1, func_integral=f, h=1/6)
print(f"Simpson 1/3 (x, y): {simpson_13_2.integral():.5f}")
print(f"Erro simples: {simpson_13_2.erro_simples():.5f}")
print(f"Erro generalizado: {simpson_13_2.erro_generalizado():.5f}")


Simpson 1/3 (x, y): 1.89637
Erro simples: 0.00001
Erro generalizado: 0.00001
